In [1]:
import numpy as np
import pandas as pd

In [2]:
watches = pd.read_csv('watches_chrono24.csv', sep = ';')

In [3]:
watches

,models,prices,countries
0,Rolex Daytona,22500.0,ES
1,Rolex GMT-Master II,61500.0,ES
2,Rolex Yacht-Master 40,13800.0,ES
3,Rolex Daytona,23500.0,ES
4,Rolex Lady-Datejust,4700.0,ES
...,...,...,...
2095,Rolex GMT-Master,28000.0,ES
2096,Rolex GMT-Master II,21576.0,UK
2097,Rolex Lady-Datejust,5656.0,US
2098,Rolex Day-Date 40,85376.0,US


### Estandarizamos todos los nombre de los modelos para poder trabajar con ellos

In [4]:
search_terms = ['GMT-Master II', 'Daytona', 'Submariner', 'Datejust', 'Day-Date', 
                'Yacht-Master II', 'Oyster Perpetual', 'Sea-Dweller', 'Explorer II', 'Milgauss', 
                'Yacht-Master', 'GMT-Master', 'Lady-Datejust', 'Explorer', 'Air King', 'Cellini', 'Precision', 'Sky-Dweller']

watches['models'] = watches['models'].replace(to_replace=r'(^.*?)('+'|'.join(search_terms)+r')(.*?$)', value='\g<2>', regex=True)

In [5]:
watches

,models,prices,countries
0,Daytona,22500.0,ES
1,GMT-Master II,61500.0,ES
2,Yacht-Master,13800.0,ES
3,Daytona,23500.0,ES
4,Lady-Datejust,4700.0,ES
...,...,...,...
2095,GMT-Master,28000.0,ES
2096,GMT-Master II,21576.0,UK
2097,Lady-Datejust,5656.0,US
2098,Day-Date,85376.0,US


In [6]:
watches['models'].value_counts().shape[0]

23

In [7]:
unique_models = watches['models'].value_counts().index.tolist()
print("Lista de modelos diferentes: ", unique_models)

Lista de modelos diferentes:  ['Datejust', 'Oyster Perpetual', 'Submariner', 'Daytona', 'GMT-Master II', 'Day-Date', 'Lady-Datejust', 'Sea-Dweller', 'Yacht-Master', 'Air King', 'GMT-Master', 'Sky-Dweller', 'Explorer II', 'Rolex', 'Cellini', 'Milgauss', 'Explorer', 'Yacht-Master II', 'Precision', 'Rolex Chronograph', 'Rolex Prince', 'Rolex Pearlmaster', 'Rolex Oyster']


### Cargamos un dataset de nombres de paises y su abreviatura

In [8]:
paises = pd.read_csv('Paises-codigo.csv')

In [9]:
paises

,nombre,name,nom,iso2,iso3,phone_code
0,Afganistán,Afghanistan,Afghanistan,AF,AFG,93
1,Albania,Albania,Albanie,AL,ALB,355
2,Alemania,Germany,Allemagne,DE,DEU,49
3,Algeria,Algeria,Algérie,DZ,DZA,213
4,Andorra,Andorra,Andorra,AD,AND,376
...,...,...,...,...,...,...
241,Wallis y Futuna,Wallis and Futuna,Wallis et Futuna,WF,WLF,681
242,Yemen,Yemen,Yémen,YE,YEM,967
243,Yibuti,Djibouti,Djibouti,DJ,DJI,253
244,Zambia,Zambia,Zambie,ZM,ZMB,260


### Cambiamos el nombre de la columna con la que vamos a trabajar, ya que se corresponde con la columna que tenemos en nuestro dataset de relojes

In [1]:
paises.rename(columns={'iso2': 'countries'}, inplace=True)

NameError: name 'paises' is not defined

In [10]:
paises = paises[['name', 'iso2']]

In [12]:
paises

,name,countries
0,Afghanistan,AF
1,Albania,AL
2,Germany,DE
3,Algeria,DZ
4,Andorra,AD
...,...,...
241,Wallis and Futuna,WF
242,Yemen,YE
243,Djibouti,DJ
244,Zambia,ZM


### Con un left join unimos ambos dataset y asi tenemos nuestros datos originales con el nombre completo de cada pais

In [13]:
watches = watches.merge(paises, on='countries', how='left')

In [14]:
watches

,models,prices,countries,name
0,Daytona,22500.0,ES,Spain
1,GMT-Master II,61500.0,ES,Spain
2,Yacht-Master,13800.0,ES,Spain
3,Daytona,23500.0,ES,Spain
4,Lady-Datejust,4700.0,ES,Spain
...,...,...,...,...
2095,GMT-Master,28000.0,ES,Spain
2096,GMT-Master II,21576.0,UK,NaN
2097,Lady-Datejust,5656.0,US,United States of America
2098,Day-Date,85376.0,US,United States of America


### Comprobamos la existencia de missing values

In [15]:
watches_na = watches[watches.isna().any(axis=1)]

In [16]:
watches_na

,models,prices,countries,name
50,GMT-Master II,NaN,ES,Spain
59,Submariner,NaN,ES,Spain
121,Daytona,NaN,ES,Spain
136,Daytona,NaN,ES,Spain
145,Day-Date,NaN,ES,Spain
...,...,...,...,...
2083,Day-Date,16328.0,UK,NaN
2089,Submariner,11371.0,UK,NaN
2092,Day-Date,64140.0,UK,NaN
2094,Datejust,7289.0,UK,NaN


### Eliminamos aquellas observaciones donde tenemos missings en el precio

In [17]:
watches = watches.dropna(subset=['prices'])

### Vemos que en las observaciones del pais UK tenemos missings en el nombre del pais porque no se ha correspondia exactamente con el nombre que aparece en el dataset que hemos cruzado. Por tanto, asignamos maualmente el nombre de "United Kingdom"

In [18]:
watches.loc[watches['countries'] == 'UK', 'name'] = 'United Kingdom'

### Cambiamos el precio a entero para trabajar de manera mas comoda con ellos

In [19]:
watches['prices'] = watches['prices'].astype(int)

C:\Users\apelaez\AppData\Local\Temp\ipykernel_7536\2837130902.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  watches['prices'] = watches['prices'].astype(int)


In [20]:
watches

,models,prices,countries,name
0,Daytona,22500,ES,Spain
1,GMT-Master II,61500,ES,Spain
2,Yacht-Master,13800,ES,Spain
3,Daytona,23500,ES,Spain
4,Lady-Datejust,4700,ES,Spain
...,...,...,...,...
2095,GMT-Master,28000,ES,Spain
2096,GMT-Master II,21576,UK,United Kingdom
2097,Lady-Datejust,5656,US,United States of America
2098,Day-Date,85376,US,United States of America


In [21]:
watches['name'].value_counts().index.tolist()

['Spain',
 'United States of America',
 'Germany',
 'Italy',
 'United Kingdom',
 'Austria',
 'France',
 'Greece',
 'Netherlands',
 'Australia',
 'Poland',
 'Romania',
 'Switzerland',
 'Japan',
 'Hungary',
 'Finland']

In [22]:
watches

,models,prices,countries,name
0,Daytona,22500,ES,Spain
1,GMT-Master II,61500,ES,Spain
2,Yacht-Master,13800,ES,Spain
3,Daytona,23500,ES,Spain
4,Lady-Datejust,4700,ES,Spain
...,...,...,...,...
2095,GMT-Master,28000,ES,Spain
2096,GMT-Master II,21576,UK,United Kingdom
2097,Lady-Datejust,5656,US,United States of America
2098,Day-Date,85376,US,United States of America


In [223]:
watches.dtypes

models       object
prices        int32
countries    object
name         object
latitude     object
longitude    object
dtype: object

### Guardamos el dataset en un csv para poder trabajar con él

In [23]:
watches.to_csv('watches.csv', index=False)